# 目的
MT4でhstファイルからエクスポートされて得られるcsvファイルを読み込み、ラベリングする。

In [1]:
from FX.FX import analyzefuncs as afs
import datetime

Using TensorFlow backend.


## データの読み込み

In [2]:
fpath = "../data/FXTF/USDJPY-cd1_20170806.csv"
data = pd.read_csv(fpath, index_col=None)
data.head()
# data.tail()

,date,time,open,high,low,close,volume
0,2017.06.26,11:47,111.657,111.660,111.646,111.646,46
1,2017.06.26,11:48,111.644,111.650,111.640,111.646,59
2,2017.06.26,11:49,111.645,111.649,111.633,111.640,67
3,2017.06.26,11:50,111.641,111.660,111.638,111.658,51
4,2017.06.26,11:51,111.656,111.659,111.652,111.653,44


## 曜日の確認
FXTFでは平日のみが保存されるようになっている。

In [5]:
count = 0
now_weekday = None
now_date = None
datetime_fmt = "%Y.%m.%d %H:%M"
for date_t, time_t in zip(data["date"], data["time"]):
    datetime_str = date_t + " " + time_t
    if now_date is None or now_date != date_t:
        now_date = date_t
        print(now_date)
    datetime_t = datetime.datetime.strptime(datetime_str, datetime_fmt)
    if now_weekday is None:
        print(datetime_t, datetime_t.weekday())
        now_weekday = datetime_t.weekday()
    elif now_weekday != datetime_t.weekday():
        print(datetime_t, datetime_t.weekday())
        now_weekday = datetime_t.weekday()
        count += 1
    if count == 7: break

2017.06.26
2017-06-26 11:47:00 0
2017.06.27
2017-06-27 00:00:00 1
2017.06.28
2017-06-28 00:00:00 2
2017.06.29
2017-06-29 00:00:00 3
2017.06.30
2017-06-30 00:00:00 4
2017.07.03
2017-07-03 00:00:00 0
2017.07.04
2017-07-04 00:00:00 1
2017.07.05
2017-07-05 00:00:00 2


## 端のデータの削除
ここでいう「端のデータ」とは、各週のデータのうち端に相当するものを指す。   
削除対象は、SMAの幅と予測したい未来の時間と現在の時間の差のうち最大の分だけのデータである。   
次の手順で削除する。   

1. データを週ごとの列に分離する。
1. 処理を施す。
1. 各列の最初と最後からデータを削除する。

In [11]:
k = 10
res = afs.makeDataset(data, 0.003, k)

In [12]:
res.to_csv(fpath.replace(".csv", "_k{0:02d}.csv".format(k)))

In [10]:
res.head(10)

,date,time,open,high,low,close,sma07,sma13,sma25,volume,label1,label2,label3
0,2017.06.26,12:12,111.685,111.687,111.676,111.685,111.689,111.688,111.669,55,1,0,0
1,2017.06.26,12:13,111.683,111.696,111.677,111.693,111.689,111.691,111.671,61,0,0,1
2,2017.06.26,12:14,111.696,111.699,111.685,111.688,111.688,111.692,111.673,44,0,1,0
3,2017.06.26,12:15,111.684,111.702,111.684,111.702,111.69,111.693,111.675,44,0,0,1
4,2017.06.26,12:16,111.701,111.711,111.698,111.711,111.693,111.694,111.677,32,0,0,1
5,2017.06.26,12:17,111.71,111.711,111.699,111.702,111.695,111.694,111.679,44,0,0,1
6,2017.06.26,12:18,111.703,111.705,111.696,111.699,111.697,111.694,111.681,36,0,0,1
7,2017.06.26,12:19,111.701,111.701,111.693,111.695,111.699,111.694,111.683,39,0,0,1
8,2017.06.26,12:20,111.693,111.712,111.692,111.711,111.701,111.695,111.685,52,0,0,1
9,2017.06.26,12:21,111.712,111.713,111.705,111.705,111.704,111.696,111.687,45,0,0,1


In [6]:
data[25:35]

,date,time,open,high,low,close,volume
25,2017.06.26,12:12,111.685,111.687,111.676,111.685,55
26,2017.06.26,12:13,111.683,111.696,111.677,111.693,61
27,2017.06.26,12:14,111.696,111.699,111.685,111.688,44
28,2017.06.26,12:15,111.684,111.702,111.684,111.702,44
29,2017.06.26,12:16,111.701,111.711,111.698,111.711,32
30,2017.06.26,12:17,111.710,111.711,111.699,111.702,44
31,2017.06.26,12:18,111.703,111.705,111.696,111.699,36
32,2017.06.26,12:19,111.701,111.701,111.693,111.695,39
33,2017.06.26,12:20,111.693,111.712,111.692,111.711,52
34,2017.06.26,12:21,111.712,111.713,111.705,111.705,45
